In [1]:
import cudf
import dask.array as da
from cuml.linear_model import LogisticRegression
from cuml.preprocessing.model_selection import train_test_split
from sklearn.datasets import load_iris

import pandas as pd
import optuna

In [2]:
def objective(trial):
    iris = load_iris()
    X, y = cudf.DataFrame(pd.DataFrame(iris.data.astype('float32'))), cudf.DataFrame(pd.DataFrame(iris.target.astype('float32')))
    solver = trial.suggest_categorical("solver", ["qn"])
    C = trial.suggest_uniform("C", 0.0, 1.0)

    if solver == "qn":
        penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
    else:
        # 'penalty' parameter isn't relevant for this solver,
        # so we always specify 'l2' as the dummy value.
        penalty = "l2"

    classifier = LogisticRegression(max_iter=200, solver=solver, C=C, penalty=penalty)

    X_train, X_valid, y_train, y_valid = train_test_split(X, y)
    classifier.fit(X_train, y_train)

    score = classifier.score(X_valid, y_valid)
    return score

In [3]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-06-10 19:24:27,151] Finished trial#0 with value: 0.7368420958518982 with parameters: {'solver': 'qn', 'C': 0.0678145987233475, 'penalty': 'l2'}. Best is trial#0 with value: 0.7368420958518982.
[I 2020-06-10 19:24:27,252] Finished trial#1 with value: 0.8421052694320679 with parameters: {'solver': 'qn', 'C': 0.7270276264196817, 'penalty': 'l2'}. Best is trial#1 with value: 0.8421052694320679.
[I 2020-06-10 19:24:27,354] Finished trial#2 with value: 0.8421052694320679 with parameters: {'solver': 'qn', 'C': 0.8375729002505089, 'penalty': 'l2'}. Best is trial#1 with value: 0.8421052694320679.
[I 2020-06-10 19:24:27,480] Finished trial#3 with value: 0.28947368264198303 with parameters: {'solver': 'qn', 'C': 0.11292021151514342, 'penalty': 'l1'}. Best is trial#1 with value: 0.8421052694320679.
[I 2020-06-10 19:24:27,568] Finished trial#4 with value: 0.28947368264198303 with parameters: {'solver': 'qn', 'C': 0.6944430510944374, 'penalty': 'l1'}. Best is trial#1 with value: 0.8421052694

Number of finished trials:  100
Best trial:
  Value:  0.9736841917037964
  Params: 
    solver: qn
    C: 0.5089016964558896
    penalty: l2
